# Following the Fundamental as T increases

For any of the configurations of current interest (N2 N3), a large number of modes we don't want enter as the thickness of the buffer air region increases.  We here try to get a better method to keep hold of the fundamental we want.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cmasher as cmr

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf, plotlogf_real
from ngsolve.webgui import Draw
from ngsolve import CF
from scipy.optimize import newton
from ipywidgets import interactive, FloatSlider, Layout

Not really sure what I'll do.  Seems like the minimization along the real is good, but there are issues there still.  Which region to I pick to look?  What realm of the cutoff?

I'm primarily trying to get good data for N2 right now with the standard buffer air thickness, and for that the fundamental is always the closest to k_low.  So that's why I'd think I could do this.  Idea is take a region we know has fundamental, check along real and find local mins, then find which is closes to k_low.  Then maybe to the min along the imaginary.

In [ ]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d]

n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]

mats = ['air', 'glass', 'air']

maxhs = [1, 1, 1]

A = BraggExact(ts=ts, mats=mats, ns=ns, maxhs=maxhs, no_mesh=True)


In [ ]:
nu = 1

def det_plot(T, wl):

    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d,]
    A.wavelength = wl * 1e-6
    A.ts = ts
        
    cutoff = A.ks[0] * A.scale
    L, R = .9995, 1.0001
    imin, imax = -.015, .001
    Xr = np.linspace(L * cutoff , R * cutoff , num=75)
    Xi = np.linspace(imin, imax, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig = plt.figure(figsize=(17,16))
    gs = fig.add_gridspec(3, 1, wspace=0.1, 
                         height_ratios=[1.5, 1,1]
                         )
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])
    ax3 = fig.add_subplot(gs[2])
    
    fs1 = A.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    ax1.set_facecolor('grey')
#     ax1.set_yticks([0])
    ax1.set_xticks([])

    xr2 = np.linspace(L * cutoff , R * cutoff , num=1000)
    xi2 = np.linspace(imin, imax, num=500)
    
    h = xr2[1] - xr2[0]                         # h for estimating derivatives
    fs = A.determinant(xr2, nu=nu, outer='h2') # function values
    fsa = np.abs(fs)                          # abs of values
    
    fsp = (fsa[2:] - fsa[:-2])/(2*h)          # center difference estimate for derivative of abs(fs)
    fspp = (fsa[2:] - 2 * fsa[1:-1] + fsa[:-2])/(h**2)  # fourth order accurate second derivative difference estimate

    dp = fsp[1:] * fsp[:-1]         # find where sign of derivative changes
    indices = np.where((dp <= 0)*(fspp[:-1]>0))[0]  # great idea from stack overflow: https://stackoverflow.com/questions/45174182/finding-all-roots-in-a-given-interval-in-1d
    indices = indices[np.where(xr2[indices]<cutoff)]
    
    if len(indices) == 0:  # if the above didn't work, try along imag near cutoff
        s = .999999
        fs = A.determinant(s * cutoff + xi2 * 1j, nu=nu, outer='h2')
        fsa = np.abs(fs) 

        fsp = (fsa[2:] - fsa[:-2])/(2*h)          
        fspp = (fsa[2:] - 2 * fsa[1:-1] + fsa[:-2])/(h**2) 

        dp = fsp[1:] * fsp[:-1]
        indices = np.where((dp <= 0)*(fspp[:-1]>0))[0]
        
        ax2.plot(xi2, np.log(fsa))
        ax2.set_title('Imaginary slice near cutoff')

        if len(indices) == 0:
            raise ValueError()
        for idx in indices:  # graph vertical lines at zeros of derivative
            m, M = ax2.get_ylim()
            ax2.plot([xi2[idx], xi2[idx]], [m,M])
            
        minxi = xi2[np.argmin(fsa)]
        fs2 = A.determinant(xr2 + 1j* minxi, nu=nu, outer='h2')
        fs2a = np.abs(fs2)
        minxr = xr2[np.argmin(fs2a)]
        ax3.plot(xr2, np.log(fs2a))   # derivatives exist at interior points xr[1:-1]
        ax3.set_title('Real slice at minimizer of imaginary slice')
        
    else:
        ax2.plot(xr2, np.log(fsa))
        ax2.set_title('Real slice')

        for idx in indices:  # graph vertical lines at zeros of derivative
            m, M = ax2.get_ylim()
            ax2.plot([xr2[idx], xr2[idx]], [m,M])
    
        minxr = xr2[max(indices[np.where(xr2[indices]<cutoff)])]
        fs2 = A.determinant(minxr + 1j* xi2, nu=nu, outer='h2')
        fs2a = np.abs(fs2)
        minxi = xi2[np.argmin(fs2a)]
        ax3.plot(xi2, np.log(fs2a))   # derivatives exist at interior points xr[1:-1]
        ax3.set_title('Imaginary slice at minimizer of real slice')

    ax2.grid('both', axis='both')
    ax3.grid('both', axis='both')
    ax1.plot(minxr, minxi, 'ro')
 
    plt.show()
    
interactive_plot = interactive(det_plot, T=FloatSlider(min=1,max=2, step=.005, value=1, 
                                                       readout_format='.3f', layout=Layout(width='100%')),
                              wl=FloatSlider(min=1.879, max=1.9, step=.0001, value=1.88, 
                                             readout_format='.6f', layout=Layout(width='100%')))
output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

# So that's sort of a plan and execution.

Let's try it on the real thing

In [ ]:
n = 300
wls = np.linspace(1.4e-6, 2e-6, n+1)
betas1 = np.zeros_like(wls, dtype=complex)
outer = 'h2'
nu = 1

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air]

ts = [4.0775e-05, 1e-5, 4.0775e-05]

mats = ['air', 'glass', 'air']

maxhs = [1, 1, 1]

In [ ]:
wl = 1.8e-6

A = BraggExact(ts=ts, maxhs=maxhs, wl=wl, mats=mats, ns=ns, no_mesh=True)

k_low = A.ks[0] * A.scale
guess = np.array(.999999 * k_low)
